In [102]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import statsmodels.api as sm

In [103]:
# Load the CSV file
csv_file_path = '/Users/orenw/Documents/tb_skills_analysis/data/SDF_UserStudy_Data/2d diameter_TLX_summary.csv'
df = pd.read_csv(csv_file_path)

In [115]:
# Define experts and non-experts
non_experts = df[df['level_training'].isin([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])]
experts = df[df['level_training'].isin([11, 12, 13, 14, 15])]

# Define assisted vs. non-assisted
non_assisted = df[df['assist_mode'] == 'baseline']
assisted = df[df['assist_mode'].isin(['haptic', 'visual', 'audio'])]
haptic = df[df['assist_mode'] == 'haptic']
visual = df[df['assist_mode'] == 'visual']
audio = df[df['assist_mode'] == 'audio']

non_assist_head = non_assisted.head(10)
assist_head = assisted.head(10)
haptic_head = haptic.head(10)
visual_head = visual.head(10)
audio_head = audio.head(10)
print(haptic_head)

                                             filepath  \
5   /Users/orenw/Documents/tb_skills_analysis/data...   
8   /Users/orenw/Documents/tb_skills_analysis/data...   
12  /Users/orenw/Documents/tb_skills_analysis/data...   
14  /Users/orenw/Documents/tb_skills_analysis/data...   
17  /Users/orenw/Documents/tb_skills_analysis/data...   
21  /Users/orenw/Documents/tb_skills_analysis/data...   
24  /Users/orenw/Documents/tb_skills_analysis/data...   
31  /Users/orenw/Documents/tb_skills_analysis/data...   
32  /Users/orenw/Documents/tb_skills_analysis/data...   
37  /Users/orenw/Documents/tb_skills_analysis/data...   

    diameter_right_eye_mean  diameter_right_eye_std  diameter_left_eye_mean  \
5                 23.031267               10.368124               18.724328   
8                 24.394440                8.360686               23.263609   
12                19.031537                4.180470               17.099911   
14                34.511622               17.954629     

## Correlating between Pupilometry and TLX

In [125]:
# Select the relevant columns from your DataFrame
correlation_df = df[['diameter_right_eye_mean', 'diameter_left_eye_mean', 'diameter_both_eyes_mean', 
                     'diameter_right_eye_std', 'diameter_left_eye_std', 'diameter_both_eyes_std', 
                     'mental', 'physical', 'temporal', 'performance', 'effort', 'frustration']]

def calculate_pvalues(df, method='pearson'):
    """Calculate correlation coefficients and p-values for a DataFrame."""
    df_cols = df.columns
    n = len(df_cols)
    pvalues = pd.DataFrame(np.zeros((n, n)), columns=df_cols, index=df_cols)
    corr_matrix = df.corr(method=method)
    
    for i in range(n):
        for j in range(n):
            if i == j:
                pvalues.iloc[i, j] = np.nan  # No p-value for self-correlation
            else:
                x = df.iloc[:, i]
                y = df.iloc[:, j]
                if method == 'pearson':
                    _, p = stats.pearsonr(x, y)
                elif method == 'spearman':
                    _, p = stats.spearmanr(x, y)
                else:
                    raise ValueError("Method must be 'pearson' or 'spearman'")
                pvalues.iloc[i, j] = p
    return corr_matrix, pvalues

# Calculate Pearson's correlation and p-values
pearson_corr_matrix, pearson_pvalues = calculate_pvalues(correlation_df, method='pearson')
print("Pearson Correlation Matrix:")
print(pearson_corr_matrix)

# Format p-values to display up to 4 decimal places without scientific notation
formatted_pearson_pvalues = pearson_pvalues.applymap(lambda x: '{0:.6f}'.format(x) if pd.notnull(x) else '')
print("\nPearson p-values:")
print(formatted_pearson_pvalues)

# Calculate Spearman's correlation and p-values
spearman_corr_matrix, spearman_pvalues = calculate_pvalues(correlation_df, method='spearman')
print("\nSpearman Correlation Matrix:")
print(spearman_corr_matrix)

# Format p-values for Spearman correlation
formatted_spearman_pvalues = spearman_pvalues.applymap(lambda x: '{0:.6f}'.format(x) if pd.notnull(x) else '')
print("\nSpearman p-values:")
print(formatted_spearman_pvalues)

Pearson Correlation Matrix:
                         diameter_right_eye_mean  diameter_left_eye_mean  \
diameter_right_eye_mean                 1.000000                0.348291   
diameter_left_eye_mean                  0.348291                1.000000   
diameter_both_eyes_mean                 0.940037                0.636234   
diameter_right_eye_std                  0.850363                0.151779   
diameter_left_eye_std                  -0.031099                0.406375   
diameter_both_eyes_std                  0.754577                0.276422   
mental                                 -0.063499                0.209584   
physical                               -0.081512                0.228268   
temporal                                0.213614                0.303438   
performance                             0.060481               -0.244860   
effort                                 -0.041648                0.148853   
frustration                            -0.218663            

In [106]:
## Multivariate regression with diameter avgs and stds predicting TLX score metrics
X = df[['diameter_right_eye_mean', 'diameter_left_eye_mean', 'diameter_both_eyes_mean', 'diameter_right_eye_std', 'diameter_left_eye_std', 'diameter_both_eyes_std']]  # Independent variables
Y = df['mental']  # Dependent variable

# Add a constant (intercept) to the model
X = sm.add_constant(X)

# Fit the multiple regression model
model = sm.OLS(Y, X).fit()

# Print out the model summary
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                 mental   R-squared:                       0.152
Model:                            OLS   Adj. R-squared:                  0.088
Method:                 Least Squares   F-statistic:                     2.369
Date:                Mon, 14 Oct 2024   Prob (F-statistic):             0.0372
Time:                        11:12:13   Log-Likelihood:                -237.56
No. Observations:                  86   AIC:                             489.1
Df Residuals:                      79   BIC:                             506.3
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                     

/Users/orenw/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  x = pd.concat(x[::order], 1)


In [110]:
X = df[['diameter_right_eye_mean', 'diameter_left_eye_mean', 'diameter_both_eyes_mean', 'diameter_right_eye_std', 'diameter_left_eye_std', 'diameter_both_eyes_std']]  # Independent variables
Y = df['physical']  # Dependent variable

# Add a constant (intercept) to the model
X = sm.add_constant(X)

# Fit the multiple regression model
model = sm.OLS(Y, X).fit()

# Print out the model summary
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:               physical   R-squared:                       0.139
Model:                            OLS   Adj. R-squared:                  0.074
Method:                 Least Squares   F-statistic:                     2.132
Date:                Mon, 14 Oct 2024   Prob (F-statistic):             0.0588
Time:                        11:14:08   Log-Likelihood:                -245.87
No. Observations:                  86   AIC:                             505.7
Df Residuals:                      79   BIC:                             522.9
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                     

/Users/orenw/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  x = pd.concat(x[::order], 1)


In [111]:
X = df[['diameter_right_eye_mean', 'diameter_left_eye_mean', 'diameter_both_eyes_mean', 'diameter_right_eye_std', 'diameter_left_eye_std', 'diameter_both_eyes_std']]  # Independent variables
Y = df['temporal']  # Dependent variable

# Add a constant (intercept) to the model
X = sm.add_constant(X)

# Fit the multiple regression model
model = sm.OLS(Y, X).fit()

# Print out the model summary
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:               temporal   R-squared:                       0.172
Model:                            OLS   Adj. R-squared:                  0.109
Method:                 Least Squares   F-statistic:                     2.730
Date:                Mon, 14 Oct 2024   Prob (F-statistic):             0.0184
Time:                        11:14:11   Log-Likelihood:                -225.63
No. Observations:                  86   AIC:                             465.3
Df Residuals:                      79   BIC:                             482.4
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                     

/Users/orenw/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  x = pd.concat(x[::order], 1)


In [112]:
X = df[['diameter_right_eye_mean', 'diameter_left_eye_mean', 'diameter_both_eyes_mean', 'diameter_right_eye_std', 'diameter_left_eye_std', 'diameter_both_eyes_std']]  # Independent variables
Y = df['performance']  # Dependent variable

# Add a constant (intercept) to the model
X = sm.add_constant(X)

# Fit the multiple regression model
model = sm.OLS(Y, X).fit()

# Print out the model summary
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:            performance   R-squared:                       0.211
Model:                            OLS   Adj. R-squared:                  0.151
Method:                 Least Squares   F-statistic:                     3.525
Date:                Mon, 14 Oct 2024   Prob (F-statistic):            0.00384
Time:                        11:14:14   Log-Likelihood:                -242.70
No. Observations:                  86   AIC:                             499.4
Df Residuals:                      79   BIC:                             516.6
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                     

/Users/orenw/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  x = pd.concat(x[::order], 1)


In [113]:
X = df[['diameter_right_eye_mean', 'diameter_left_eye_mean', 'diameter_both_eyes_mean', 'diameter_right_eye_std', 'diameter_left_eye_std', 'diameter_both_eyes_std']]  # Independent variables
Y = df['effort']  # Dependent variable

# Add a constant (intercept) to the model
X = sm.add_constant(X)

# Fit the multiple regression model
model = sm.OLS(Y, X).fit()

# Print out the model summary
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                 effort   R-squared:                       0.169
Model:                            OLS   Adj. R-squared:                  0.105
Method:                 Least Squares   F-statistic:                     2.670
Date:                Mon, 14 Oct 2024   Prob (F-statistic):             0.0207
Time:                        11:14:18   Log-Likelihood:                -236.88
No. Observations:                  86   AIC:                             487.8
Df Residuals:                      79   BIC:                             504.9
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                     

/Users/orenw/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  x = pd.concat(x[::order], 1)


In [114]:
X = df[['diameter_right_eye_mean', 'diameter_left_eye_mean', 'diameter_both_eyes_mean', 'diameter_right_eye_std', 'diameter_left_eye_std', 'diameter_both_eyes_std']]  # Independent variables
Y = df['frustration']  # Dependent variable

# Add a constant (intercept) to the model
X = sm.add_constant(X)

# Fit the multiple regression model
model = sm.OLS(Y, X).fit()

# Print out the model summary
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:            frustration   R-squared:                       0.087
Model:                            OLS   Adj. R-squared:                  0.018
Method:                 Least Squares   F-statistic:                     1.255
Date:                Mon, 14 Oct 2024   Prob (F-statistic):              0.288
Time:                        11:14:57   Log-Likelihood:                -265.76
No. Observations:                  86   AIC:                             545.5
Df Residuals:                      79   BIC:                             562.7
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                     

/Users/orenw/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  x = pd.concat(x[::order], 1)


In [135]:

X = df[['mental', 'physical', 'temporal', 'performance', 'effort', 'frustration']]  # Independent variables
Y = df['diameter_both_eyes_mean']  # Dependent variable

# Add a constant (intercept) to the model
X = sm.add_constant(X)

# Fit the multiple regression model
model = sm.OLS(Y, X).fit()

# Print out the model summary
print(model.summary())

# Print p-values with more decimal precision
print("\nP-values with more precision:")
print(model.pvalues.apply(lambda x: f"{x:.10f}"))

                               OLS Regression Results                              
Dep. Variable:     diameter_both_eyes_mean   R-squared:                       0.232
Model:                                 OLS   Adj. R-squared:                  0.174
Method:                      Least Squares   F-statistic:                     3.988
Date:                     Mon, 14 Oct 2024   Prob (F-statistic):            0.00155
Time:                             19:53:43   Log-Likelihood:                -217.37
No. Observations:                       86   AIC:                             448.7
Df Residuals:                           79   BIC:                             465.9
Df Model:                                6                                         
Covariance Type:                 nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------


/Users/orenw/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  x = pd.concat(x[::order], 1)


In [132]:
X = df[['mental', 'physical', 'temporal', 'performance', 'effort', 'frustration']]  # Independent variables
Y = df['diameter_both_eyes_std']  # Dependent variable

# Add a constant (intercept) to the model
X = sm.add_constant(X)

# Fit the multiple regression model
model = sm.OLS(Y, X).fit()

# Print out the model summary
print(model.summary())

                              OLS Regression Results                              
Dep. Variable:     diameter_both_eyes_std   R-squared:                       0.084
Model:                                OLS   Adj. R-squared:                  0.015
Method:                     Least Squares   F-statistic:                     1.209
Date:                    Mon, 14 Oct 2024   Prob (F-statistic):              0.310
Time:                            19:51:25   Log-Likelihood:                -236.39
No. Observations:                      86   AIC:                             486.8
Df Residuals:                          79   BIC:                             504.0
Df Model:                               6                                         
Covariance Type:                nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const     

/Users/orenw/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  x = pd.concat(x[::order], 1)


## Assisted vs. Non-assisted

## TLX

In [87]:
## TLX results assist modes
avg_mental_non_assisted = non_assisted['mental'].mean()
avg_mental_assisted = assisted['mental'].mean()
avg_mental_haptic = haptic['mental'].mean()
avg_mental_visual = visual['mental'].mean()
avg_mental_audio = audio['mental'].mean()

avg_physical_non_assisted = non_assisted['physical'].mean()
avg_physical_assisted = assisted['physical'].mean()
avg_physical_haptic = haptic['physical'].mean()
avg_physical_visual = visual['physical'].mean()
avg_physical_audio = audio['physical'].mean()

avg_temporal_non_assisted = non_assisted['temporal'].mean()
avg_temporal_assisted = assisted['temporal'].mean()
avg_temporal_haptic = haptic['temporal'].mean()
avg_temporal_visual = visual['temporal'].mean()
avg_temporal_audio = audio['temporal'].mean()

avg_performance_non_assisted = non_assisted['performance'].mean()
avg_performance_assisted = assisted['performance'].mean()
avg_performance_haptic = haptic['performance'].mean()
avg_performance_visual = visual['performance'].mean()
avg_performance_audio = audio['performance'].mean()

avg_effort_non_assisted = non_assisted['effort'].mean()
avg_effort_assisted = assisted['effort'].mean()
avg_effort_haptic = haptic['effort'].mean()
avg_effort_visual = visual['effort'].mean()
avg_effort_audio = audio['effort'].mean()

avg_frustration_non_assisted = non_assisted['frustration'].mean()
avg_frustration_assisted = assisted['frustration'].mean()
avg_frustration_haptic = haptic['frustration'].mean()
avg_frustration_visual = visual['frustration'].mean()
avg_frustration_audio = audio['frustration'].mean()

print(f"Average mental of non-assisted vs. assisted: {avg_mental_non_assisted} vs. {avg_mental_assisted}")
print(f"Average physical of non-assisted vs. assisted: {avg_physical_non_assisted} vs. {avg_physical_assisted}")
print(f"Average temporal of non-assisted vs. assisted: {avg_temporal_non_assisted} vs. {avg_temporal_assisted}")
print(f"Average performance of non-assisted vs. assisted: {avg_performance_non_assisted} vs. {avg_performance_assisted}")
print(f"Average effort of non-assisted vs. assisted: {avg_effort_non_assisted} vs. {avg_effort_assisted}")
print(f"Average frustration of non-assisted vs. assisted: {avg_frustration_non_assisted} vs. {avg_frustration_assisted}")

Average mental of non-assisted vs. assisted: 8.962962962962964 vs. 8.576271186440678
Average physical of non-assisted vs. assisted: 7.555555555555555 vs. 6.779661016949152
Average temporal of non-assisted vs. assisted: 6.2592592592592595 vs. 5.47457627118644
Average performance of non-assisted vs. assisted: 11.592592592592593 vs. 13.389830508474576
Average effort of non-assisted vs. assisted: 9.444444444444445 vs. 7.983050847457627
Average frustration of non-assisted vs. assisted: 9.25925925925926 vs. 6.610169491525424


In [89]:
## TLX results assist modes stats
# T tests
# Define the TLX criteria
tlx_criteria = ['mental', 'physical', 'temporal', 'performance', 'effort', 'frustration']

# Initialize an empty dictionary to store t-test results
t_test_results = {}

# Non-assisted vs. assisted
for criterion in tlx_criteria:
    t_stat, p_value = stats.ttest_ind(non_assisted[criterion], assisted[criterion], equal_var=True)
    t_test_results[criterion] = (t_stat, p_value)
for criterion, (t_stat, p_value) in t_test_results.items():
    print(f"T-test for {criterion} non-assisted vs. assisted: t-stat = {t_stat}, p-value = {p_value}")

# Non-assisted vs. haptic
for criterion in tlx_criteria:
    t_stat, p_value = stats.ttest_ind(non_assisted[criterion], haptic[criterion], equal_var=True)
    t_test_results[criterion] = (t_stat, p_value)
for criterion, (t_stat, p_value) in t_test_results.items():
    print(f"T-test for {criterion} non-assisted vs. haptic: t-stat = {t_stat}, p-value = {p_value}")

# Non-assisted vs. visual
for criterion in tlx_criteria:
    t_stat, p_value = stats.ttest_ind(non_assisted[criterion], visual[criterion], equal_var=True)
    t_test_results[criterion] = (t_stat, p_value)
for criterion, (t_stat, p_value) in t_test_results.items():
    print(f"T-test for {criterion} non-assisted vs. visual: t-stat = {t_stat}, p-value = {p_value}")

# Non-assisted vs. audio
for criterion in tlx_criteria:
    t_stat, p_value = stats.ttest_ind(non_assisted[criterion], audio[criterion], equal_var=True)
    t_test_results[criterion] = (t_stat, p_value)
for criterion, (t_stat, p_value) in t_test_results.items():
    print(f"T-test for {criterion} non-assisted vs. audio: t-stat = {t_stat}, p-value = {p_value}")

T-test for mental non-assisted vs. assisted: t-stat = 0.39550663213460496, p-value = 0.6934717059693059
T-test for physical non-assisted vs. assisted: t-stat = 0.7277150521907457, p-value = 0.4688116158492084
T-test for temporal non-assisted vs. assisted: t-stat = 0.9151643050181842, p-value = 0.3627250711485417
T-test for performance non-assisted vs. assisted: t-stat = -1.6973127208241316, p-value = 0.09333990118170327
T-test for effort non-assisted vs. assisted: t-stat = 1.5109046759953688, p-value = 0.13456562421719898
T-test for frustration non-assisted vs. assisted: t-stat = 2.0753303285420457, p-value = 0.041015104914107495
T-test for mental non-assisted vs. haptic: t-stat = 0.7399844013721542, p-value = 0.4629899307346321
T-test for physical non-assisted vs. haptic: t-stat = 0.6041181585230072, p-value = 0.5486702209079327
T-test for temporal non-assisted vs. haptic: t-stat = 0.8245091215607426, p-value = 0.4138152663144674
T-test for performance non-assisted vs. haptic: t-stat 

## Pupil Diameter

In [117]:
## Calculate averages, standard deviation, and confidence intervals for pupil diameters

def mean_confidence_interval(data, confidence=0.95):
    n = len(data)
    mean = data.mean()
    std_dev = data.std()
    stderr = stats.sem(data)
    margin_of_error = stderr * stats.t.ppf((1 + confidence) / 2., n-1)
    return mean, std_dev, mean - margin_of_error, mean + margin_of_error

# Right pupil
avg_right_non_assisted, std_right_non_assisted, right_ci_low_non_assisted, right_ci_high_non_assisted = mean_confidence_interval(non_assisted['diameter_right_eye_mean'])
avg_right_assisted, std_right_assisted, right_ci_low_assisted, right_ci_high_assisted = mean_confidence_interval(assisted['diameter_right_eye_mean'])

# Left pupil
avg_left_non_assisted, std_left_non_assisted, left_ci_low_non_assisted, left_ci_high_non_assisted = mean_confidence_interval(non_assisted['diameter_left_eye_mean'])
avg_left_assisted, std_left_assisted, left_ci_low_assisted, left_ci_high_assisted = mean_confidence_interval(assisted['diameter_left_eye_mean'])

# Average pupil diameter and std (right and left average)
non_assisted['nonassist_avg_pupil_diameter'] = non_assisted['diameter_both_eyes_mean']
non_assisted['nonassist_std_pupil_diameter'] = non_assisted['diameter_both_eyes_std']
assisted['assist_avg_pupil_diameter'] = assisted['diameter_both_eyes_mean']
assisted['assist_std_pupil_diameter'] = assisted['diameter_both_eyes_std']
haptic['haptic_avg_pupil_diameter'] = haptic['diameter_both_eyes_mean']
haptic['haptic_std_pupil_diameter'] = haptic['diameter_both_eyes_std']
visual['visual_avg_pupil_diameter'] = visual['diameter_both_eyes_mean']
visual['visual_std_pupil_diameter'] = visual['diameter_both_eyes_std']
audio['audio_avg_pupil_diameter'] = audio['diameter_both_eyes_mean']
audio['audio_std_pupil_diameter'] = audio['diameter_both_eyes_std']

avg_avg_non_assisted, std_avg_non_assisted, avg_ci_low_non_assisted, avg_ci_high_non_assisted = mean_confidence_interval(non_assisted['nonassist_avg_pupil_diameter'])
avg_avg_assisted, std_avg_assisted, avg_ci_low_assisted, avg_ci_high_assisted = mean_confidence_interval(assisted['assist_avg_pupil_diameter'])
avg_avg_haptic, std_avg_haptic, avg_ci_low_haptic, avg_ci_high_haptic = mean_confidence_interval(haptic['haptic_avg_pupil_diameter'])
avg_avg_visual, std_avg_visual, avg_ci_low_visual, avg_ci_high_visual = mean_confidence_interval(visual['visual_avg_pupil_diameter'])
avg_avg_audio, std_avg_audio, avg_ci_low_audio, avg_ci_high_audio = mean_confidence_interval(audio['audio_avg_pupil_diameter'])

# Print results with confidence intervals and standard deviations
print(f"Average right pupil of non-assisted: {avg_right_non_assisted} (SD: {std_right_non_assisted}, CI: {right_ci_low_non_assisted} - {right_ci_high_non_assisted})")
print(f"Average right pupil of assisted: {avg_right_assisted} (SD: {std_right_assisted}, CI: {right_ci_low_assisted} - {right_ci_high_assisted})")

print(f"Average left pupil of non-assisted: {avg_left_non_assisted} (SD: {std_left_non_assisted}, CI: {left_ci_low_non_assisted} - {left_ci_high_non_assisted})")
print(f"Average left pupil of assisted: {avg_left_assisted} (SD: {std_left_assisted}, CI: {left_ci_low_assisted} - {left_ci_high_assisted})")

print(f"Average average pupil of non-assisted: {avg_avg_non_assisted} (SD: {std_avg_non_assisted}, CI: {avg_ci_low_non_assisted} - {avg_ci_high_non_assisted})")
print(f"Average average pupil of assisted: {avg_avg_assisted} (SD: {std_avg_assisted}, CI: {avg_ci_low_assisted} - {avg_ci_high_assisted})")
print(f"Average average pupil of haptic: {avg_avg_haptic} (SD: {std_avg_haptic}, CI: {avg_ci_low_haptic} - {avg_ci_high_haptic})")
print(f"Average average pupil of visual: {avg_avg_visual} (SD: {std_avg_visual}, CI: {avg_ci_low_visual} - {avg_ci_high_visual})")
print(f"Average average pupil of audio: {avg_avg_audio} (SD: {std_avg_audio}, CI: {avg_ci_low_audio} - {avg_ci_high_audio})")


Average right pupil of non-assisted: 20.992027799629632 (SD: 3.8797548235612296, CI: 19.457247907550812 - 22.526807691708452)
Average right pupil of assisted: 21.195466357796608 (SD: 6.043698976566431, CI: 19.620469827257885 - 22.77046288833533)
Average left pupil of non-assisted: 19.14057583333333 (SD: 3.682308734599976, CI: 17.683903012677437 - 20.597248653989222)
Average left pupil of assisted: 18.239994839491523 (SD: 2.5663686302712625, CI: 17.571195532240615 - 18.90879414674243)
Average average pupil of non-assisted: 19.953499732222227 (SD: 2.833554575264143, CI: 18.83258287458649 - 21.074416589857965)
Average average pupil of assisted: 19.705966157457627 (SD: 3.757543572103507, CI: 18.726744969844255 - 20.685187345071)
Average average pupil of haptic: 20.536908207272727 (SD: 4.68839302984668, CI: 18.458191190775054 - 22.6156252237704)
Average average pupil of visual: 19.473938378500005 (SD: 3.255699355624463, CI: 17.95022417709574 - 20.99765257990427)
Average average pupil of aud

<ipython-input-117-6642adacd33b>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_assisted['nonassist_avg_pupil_diameter'] = non_assisted['diameter_both_eyes_mean']
<ipython-input-117-6642adacd33b>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_assisted['nonassist_std_pupil_diameter'] = non_assisted['diameter_both_eyes_std']
<ipython-input-117-6642adacd33b>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = va

In [118]:
## T-test comparing right and left eye mean diameters between non-assisted and assisted
# T-test for right pupil diameter
right_avg_t_stat, right_avg_p_value = stats.ttest_ind(non_assisted['diameter_right_eye_mean'], assisted['diameter_right_eye_mean'], equal_var=True)
right_std_t_stat, right_std_p_value = stats.ttest_ind(non_assisted['diameter_right_eye_std'], assisted['diameter_right_eye_std'], equal_var=True)

# T-test for left pupil diameter
left_avg_t_stat, left_avg_p_value = stats.ttest_ind(non_assisted['diameter_left_eye_mean'], assisted['diameter_left_eye_mean'], equal_var=True)
left_std_t_stat, left_std_p_value = stats.ttest_ind(non_assisted['diameter_left_eye_std'], assisted['diameter_left_eye_std'], equal_var=True)

# T-test for average of both eyes
avg_avg_t_stat, avg_avg_p_value = stats.ttest_ind(non_assisted['nonassist_avg_pupil_diameter'], assisted['assist_avg_pupil_diameter'], equal_var=True)
avg_std_t_stat, avg_std_p_value = stats.ttest_ind(non_assisted['nonassist_std_pupil_diameter'], assisted['assist_std_pupil_diameter'], equal_var=True)

# Printing the results
print(f"Avg right pupil diameter non-assisted vs. assisted (t-stat, p-value): {right_avg_t_stat}, {right_avg_p_value}")
print(f"Std dev right pupil diameter non-assisted vs. assisted (t-stat, p-value): {right_std_t_stat}, {right_std_p_value}")

print(f"Avg left pupil diameter non-assisted vs. assisted (t-stat, p-value): {left_avg_t_stat}, {left_avg_p_value}")
print(f"Std dev left pupil diameter non-assisted vs. assisted (t-stat, p-value): {left_std_t_stat}, {left_std_p_value}")

print(f"Avg avg pupil diameter non-assisted vs. assisted (t-stat, p-value): {avg_avg_t_stat}, {avg_avg_p_value}")
print(f"Avg std pupil diameter non-assisted vs. assisted (t-stat, p-value): {avg_std_t_stat}, {avg_std_p_value}")

Avg right pupil diameter non-assisted vs. assisted (t-stat, p-value): -0.16017849723524924, 0.8731250307743328
Std dev right pupil diameter non-assisted vs. assisted (t-stat, p-value): -0.7050961996970382, 0.4827009711769098
Avg left pupil diameter non-assisted vs. assisted (t-stat, p-value): 1.3107243600672536, 0.19352320051484795
Std dev left pupil diameter non-assisted vs. assisted (t-stat, p-value): -0.5541125393407713, 0.5809740219157296
Avg avg pupil diameter non-assisted vs. assisted (t-stat, p-value): 0.3045837477926941, 0.7614365344142267
Avg std pupil diameter non-assisted vs. assisted (t-stat, p-value): -0.9961680745843127, 0.32202949744965487


In [121]:
## Mann-Whitney U test comparing right and left eye mean diameters between non-assisted and assisted
# Mann-Whitney U test for right pupil diameter
right_avg_u_stat, right_avg_p_value = stats.mannwhitneyu(non_assisted['diameter_right_eye_mean'], assisted['diameter_right_eye_mean'], alternative='two-sided')
right_std_u_stat, right_std_p_value = stats.mannwhitneyu(non_assisted['diameter_right_eye_std'], assisted['diameter_right_eye_std'], alternative='two-sided')

# Mann-Whitney U test for left pupil diameter
left_avg_u_stat, left_avg_p_value = stats.mannwhitneyu(non_assisted['diameter_left_eye_mean'], assisted['diameter_left_eye_mean'], alternative='two-sided')
left_std_u_stat, left_std_p_value = stats.mannwhitneyu(non_assisted['diameter_left_eye_std'], assisted['diameter_left_eye_std'], alternative='two-sided')

# Mann-Whitney U test for average of both eyes
avg_avg_u_stat, avg_avg_p_value = stats.mannwhitneyu(non_assisted['nonassist_avg_pupil_diameter'], assisted['assist_avg_pupil_diameter'], alternative='two-sided')
avg_std_u_stat, avg_std_p_value = stats.mannwhitneyu(non_assisted['nonassist_std_pupil_diameter'], assisted['assist_std_pupil_diameter'], alternative='two-sided')

# Printing the results
print(f"Avg right pupil diameter non-assisted vs. assisted (u-stat, p-value): {right_avg_u_stat}, {right_avg_p_value}")
print(f"Std dev right pupil diameter non-assisted vs. assisted (u-stat, p-value): {right_std_u_stat}, {right_std_p_value}")

print(f"Avg left pupil diameter non-assisted vs. assisted (u-stat, p-value): {left_avg_u_stat}, {left_avg_p_value}")
print(f"Std dev left pupil diameter non-assisted vs. assisted (u-stat, p-value): {left_std_u_stat}, {left_std_p_value}")

print(f"Avg avg pupil diameter non-assisted vs. assisted (u-stat, p-value): {avg_avg_u_stat}, {avg_avg_p_value}")
print(f"Avg std pupil diameter non-assisted vs. assisted (u-stat, p-value): {avg_std_u_stat}, {avg_std_p_value}")

Avg right pupil diameter non-assisted vs. assisted (u-stat, p-value): 886.0, 0.4075811189609029
Std dev right pupil diameter non-assisted vs. assisted (u-stat, p-value): 805.0, 0.9406593636338003
Avg left pupil diameter non-assisted vs. assisted (u-stat, p-value): 891.0, 0.3817461093127681
Std dev left pupil diameter non-assisted vs. assisted (u-stat, p-value): 678.0, 0.2722014245955172
Avg avg pupil diameter non-assisted vs. assisted (u-stat, p-value): 881.0, 0.4344308076370943
Avg std pupil diameter non-assisted vs. assisted (u-stat, p-value): 742.0, 0.6153314729803866


In [128]:
## T-test comparing right and left eye mean diameters between non-assisted and some specific form of assistance
# T-test for right pupil diameter
right_avg_t_stat, right_avg_p_value = stats.ttest_ind(non_assisted['diameter_right_eye_mean'], audio['diameter_right_eye_mean'], equal_var=True)
right_std_t_stat, right_std_p_value = stats.ttest_ind(non_assisted['diameter_right_eye_std'], audio['diameter_right_eye_std'], equal_var=True)

# T-test for left pupil diameter
left_avg_t_stat, left_avg_p_value = stats.ttest_ind(non_assisted['diameter_left_eye_mean'], audio['diameter_left_eye_mean'], equal_var=True)
left_std_t_stat, left_std_p_value = stats.ttest_ind(non_assisted['diameter_left_eye_std'], audio['diameter_left_eye_std'], equal_var=True)

# T-test for average of both eyes
avg_avg_t_stat, avg_avg_p_value = stats.ttest_ind(non_assisted['nonassist_avg_pupil_diameter'], audio['audio_avg_pupil_diameter'], equal_var=True)
avg_std_t_stat, avg_std_p_value = stats.ttest_ind(non_assisted['nonassist_std_pupil_diameter'], audio['audio_std_pupil_diameter'], equal_var=True)

# Printing the results
print(f"Avg right pupil diameter non-assisted vs. visual (t-stat, p-value): {right_avg_t_stat}, {right_avg_p_value}")
print(f"Std dev right pupil diameter non-assisted vs. visual (t-stat, p-value): {right_std_t_stat}, {right_std_p_value}")

print(f"Avg left pupil diameter non-assisted vs. visual (t-stat, p-value): {left_avg_t_stat}, {left_avg_p_value}")
print(f"Std dev left pupil diameter non-assisted vs. visual (t-stat, p-value): {left_std_t_stat}, {left_std_p_value}")

print(f"Avg avg pupil diameter non-assisted vs. visual (t-stat, p-value): {avg_avg_t_stat}, {avg_avg_p_value}")
print(f"Avg std pupil diameter non-assisted vs. visual (t-stat, p-value): {avg_std_t_stat}, {avg_std_p_value}")

Avg right pupil diameter non-assisted vs. visual (t-stat, p-value): 1.0762955830439418, 0.28793886162083443
Std dev right pupil diameter non-assisted vs. visual (t-stat, p-value): 0.49853240769517987, 0.6207097394955613
Avg left pupil diameter non-assisted vs. visual (t-stat, p-value): 0.9761453386130869, 0.33458157568581137
Std dev left pupil diameter non-assisted vs. visual (t-stat, p-value): 0.353143994934879, 0.7257468479581527
Avg avg pupil diameter non-assisted vs. visual (t-stat, p-value): 1.1963002928634734, 0.23828928569673732
Avg std pupil diameter non-assisted vs. visual (t-stat, p-value): 0.5412750435628025, 0.59117774766374


## Expert vs. Non-expert

## TLX

In [64]:
## TLX results experts vs. non-experts means
avg_mental_non_experts = non_experts['mental'].mean()
avg_mental_experts = experts['mental'].mean()

avg_physical_non_experts = non_experts['physical'].mean()
avg_physical_experts = experts['physical'].mean()

avg_temporal_non_experts = non_experts['temporal'].mean()
avg_temporal_experts = experts['temporal'].mean()

avg_performance_non_experts = non_experts['performance'].mean()
avg_performance_experts = experts['performance'].mean()

avg_effort_non_experts = non_experts['effort'].mean()
avg_effort_experts = experts['effort'].mean()

avg_frustration_non_experts = non_experts['frustration'].mean()
avg_frustration_experts = experts['frustration'].mean()

print(f"Average mental of non-experts vs. experts: {avg_mental_non_experts} vs. {avg_mental_experts}")
print(f"Average physical of non-experts vs. experts: {avg_physical_non_experts} vs. {avg_physical_experts}")
print(f"Average temporal of non-experts vs. experts: {avg_temporal_non_experts} vs. {avg_temporal_experts}")
print(f"Average performance of non-experts vs. experts: {avg_performance_non_experts} vs. {avg_performance_experts}")
print(f"Average effort of non-experts vs. experts: {avg_effort_non_experts} vs. {avg_effort_experts}")
print(f"Average frustration of non-experts vs. experts: {avg_frustration_non_experts} vs. {avg_frustration_experts}")

Average mental of non-experts vs. experts: 8.914285714285715 vs. 7.75
Average physical of non-experts vs. experts: 7.142857142857143 vs. 6.5
Average temporal of non-experts vs. experts: 6.128571428571429 vs. 3.9375
Average performance of non-experts vs. experts: 12.085714285714285 vs. 16.0625
Average effort of non-experts vs. experts: 8.714285714285714 vs. 7.25
Average frustration of non-experts vs. experts: 7.4 vs. 7.625


In [65]:
## TLX results experts vs. non-experts stats
# T tests
# Define the TLX criteria
tlx_criteria = ['mental', 'physical', 'temporal', 'performance', 'effort', 'frustration']

# Initialize an empty dictionary to store t-test results
t_test_results = {}

# Run t-tests for each TLX criterion and store results
for criterion in tlx_criteria:
    t_stat, p_value = stats.ttest_ind(non_experts[criterion], experts[criterion], equal_var=True)
    t_test_results[criterion] = (t_stat, p_value)

# Print the t-test results
for criterion, (t_stat, p_value) in t_test_results.items():
    print(f"T-test for {criterion} non-experts vs. experts: t-stat = {t_stat}, p-value = {p_value}")

T-test for mental non-experts vs. experts: t-stat = 1.0035371457217643, p-value = 0.3184838982432044
T-test for physical non-experts vs. experts: t-stat = 0.5047393453031432, p-value = 0.6150635122690205
T-test for temporal non-experts vs. experts: t-stat = 2.192247202283919, p-value = 0.031128038634793638
T-test for performance non-experts vs. experts: t-stat = -3.28992297071687, p-value = 0.0014658032921667614
T-test for effort non-experts vs. experts: t-stat = 1.2643517369448254, p-value = 0.20960079982736912
T-test for frustration non-experts vs. experts: t-stat = -0.14416837714746353, p-value = 0.8857128347786459


## Pupil Diameter

In [129]:
## Calculate averages, standard deviation, and confidence intervals for pupil diameters

def mean_confidence_interval(data, confidence=0.95):
    n = len(data)
    mean = data.mean()
    std_dev = data.std()
    stderr = stats.sem(data)
    margin_of_error = stderr * stats.t.ppf((1 + confidence) / 2., n-1)
    return mean, std_dev, mean - margin_of_error, mean + margin_of_error

# Right pupil
avg_right_non_experts, std_right_non_experts, right_ci_low_non_experts, right_ci_high_non_experts = mean_confidence_interval(non_experts['diameter_right_eye_mean'])
avg_right_experts, std_right_experts, right_ci_low_experts, right_ci_high_experts = mean_confidence_interval(experts['diameter_right_eye_mean'])

# Left pupil
avg_left_non_experts, std_left_non_experts, left_ci_low_non_experts, left_ci_high_non_experts = mean_confidence_interval(non_experts['diameter_left_eye_mean'])
avg_left_experts, std_left_experts, left_ci_low_experts, left_ci_high_experts = mean_confidence_interval(experts['diameter_left_eye_mean'])

# Average pupil diameter and std (right and left average)
non_experts['nonexp_avg_pupil_diameter'] = non_experts['diameter_both_eyes_mean']
experts['exp_avg_pupil_diameter'] = experts['diameter_both_eyes_mean']
non_experts['nonexp_std_pupil_diameter'] = non_experts['diameter_both_eyes_std']
experts['exp_std_pupil_diameter'] = experts['diameter_both_eyes_std']

avg_avg_non_experts, std_avg_non_experts, avg_ci_low_non_experts, avg_ci_high_non_experts = mean_confidence_interval(non_experts['nonexp_avg_pupil_diameter'])
avg_avg_experts, std_avg_experts, avg_ci_low_experts, avg_ci_high_experts = mean_confidence_interval(experts['exp_avg_pupil_diameter'])

# Print results with confidence intervals and standard deviations
print(f"Average right pupil of non-experts: {avg_right_non_experts} (SD: {std_right_non_experts}, CI: {right_ci_low_non_experts} - {right_ci_high_non_experts})")
print(f"Average right pupil of experts: {avg_right_experts} (SD: {std_right_experts}, CI: {right_ci_low_experts} - {right_ci_high_experts})")

print(f"Average left pupil of non-experts: {avg_left_non_experts} (SD: {std_left_non_experts}, CI: {left_ci_low_non_experts} - {left_ci_high_non_experts})")
print(f"Average left pupil of experts: {avg_left_experts} (SD: {std_left_experts}, CI: {left_ci_low_experts} - {left_ci_high_experts})")

print(f"Average average pupil of non-experts: {avg_avg_non_experts} (SD: {std_avg_non_experts}, CI: {avg_ci_low_non_experts} - {avg_ci_high_non_experts})")
print(f"Average average pupil of experts: {avg_avg_experts} (SD: {std_avg_experts}, CI: {avg_ci_low_experts} - {avg_ci_high_experts})")


Average right pupil of non-experts: 20.035086285428573 (SD: 3.9947568428465345, CI: 19.082570138670224 - 20.98760243218692)
Average right pupil of experts: 25.9288266075 (SD: 7.988203965536489, CI: 21.672213179455518 - 30.18544003554448)
Average left pupil of non-experts: 18.391035702714284 (SD: 2.6885937909988793, CI: 17.74996314216869 - 19.03210826325988)
Average left pupil of experts: 19.09892149 (SD: 4.035467151730053, CI: 16.948572833331347 - 21.24927014666865)
Average average pupil of non-experts: 19.201316917571432 (SD: 3.1876563880669293, CI: 18.4412470811837 - 19.961386753959165)
Average average pupil of experts: 22.331519489375 (SD: 3.6521767290702694, CI: 20.38541188200521 - 24.27762709674479)


<ipython-input-129-cd194ae4cab8>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_experts['nonexp_avg_pupil_diameter'] = non_experts['diameter_both_eyes_mean']
<ipython-input-129-cd194ae4cab8>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  experts['exp_avg_pupil_diameter'] = experts['diameter_both_eyes_mean']
<ipython-input-129-cd194ae4cab8>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the

In [130]:
## T-test comparing right and left eye mean diameters between non_experts and experts
# T-test for right pupil diameter
right_avg_t_stat, right_avg_p_value = stats.ttest_ind(non_experts['diameter_right_eye_mean'], experts['diameter_right_eye_mean'], equal_var=True)
right_std_t_stat, right_std_p_value = stats.ttest_ind(non_experts['diameter_right_eye_std'], experts['diameter_right_eye_std'], equal_var=True)

# T-test for left pupil diameter
left_avg_t_stat, left_avg_p_value = stats.ttest_ind(non_experts['diameter_left_eye_mean'], experts['diameter_left_eye_mean'], equal_var=True)
left_std_t_stat, left_std_p_value = stats.ttest_ind(non_experts['diameter_left_eye_std'], experts['diameter_left_eye_std'], equal_var=True)

# T-test for average of both eyes
avg_avg_t_stat, avg_avg_p_value = stats.ttest_ind(non_experts['nonexp_avg_pupil_diameter'], experts['exp_avg_pupil_diameter'], equal_var=True)
avg_std_t_stat, avg_std_p_value = stats.ttest_ind(non_experts['nonexp_std_pupil_diameter'], experts['exp_std_pupil_diameter'], equal_var=True)

# Printing the results
print(f"Avg right pupil diameter non-experts vs. experts (t-stat, p-value): {right_avg_t_stat}, {right_avg_p_value}")
print(f"Std dev right pupil diameter non-experts vs. experts (t-stat, p-value): {right_std_t_stat}, {right_std_p_value}")

print(f"Avg left pupil diameter non-experts vs. experts (t-stat, p-value): {left_avg_t_stat}, {left_avg_p_value}")
print(f"Std dev left pupil diameter non-experts vs. experts (t-stat, p-value): {left_std_t_stat}, {left_std_p_value}")

print(f"Avg avg pupil diameter non-experts vs. experts (t-stat, p-value): {avg_avg_t_stat}, {avg_avg_p_value}")
print(f"Avg std pupil diameter non-experts vs. experts (t-stat, p-value): {avg_std_t_stat}, {avg_std_p_value}")

Avg right pupil diameter non-experts vs. experts (t-stat, p-value): -4.29673267402442, 4.636447860329283e-05
Std dev right pupil diameter non-experts vs. experts (t-stat, p-value): -4.201537222050501, 6.575604053707524e-05
Avg left pupil diameter non-experts vs. experts (t-stat, p-value): -0.8589252503759596, 0.3928262306931777
Std dev left pupil diameter non-experts vs. experts (t-stat, p-value): -2.2390175188398356, 0.02779812829656365
Avg avg pupil diameter non-experts vs. experts (t-stat, p-value): -3.4487551828281715, 0.0008828273733849216
Avg std pupil diameter non-experts vs. experts (t-stat, p-value): -5.6399175838022, 2.2348613662752334e-07
